In [1]:
%load_ext lab_black
%cd ..

/home/shim/cev/dl/log-analytics


In [50]:
import pandas as pd
from datasets.dataset_ver5 import MyDatasetVer5
from tqdm import tqdm
import sys
from collections import defaultdict

In [3]:
df = pd.read_csv("data/ori/train.csv")

In [4]:
# {텍스트: 레벨: 개수}
db = {}

In [6]:
with tqdm(total=len(df), ncols=100, file=sys.stdout) as t:
    for i, (level, full_log) in enumerate(zip(df.level, df.full_log)):
        text = MyDatasetVer5.refine_data(full_log)
        if text not in db:
            db[text] = {}
        if level not in db[text]:
            db[text][level] = {"cnt": 0, "list": []}
        db[text][level]["cnt"] += 1
        db[text][level]["list"].append(i)

        t.update()

100%|████████████████████████████████████████████████████| 472972/472972 [00:42<00:00, 11111.45it/s]


In [7]:
keys = list(db.keys())

In [34]:
len(keys), len(df)

(152791, 472972)

In [31]:
duples = {}
for i, key in enumerate(keys):
    if len(db[key]) != 1:
        K = tuple(sorted(list(db[key].keys())))
        if K not in duples:
            duples[K] = []
        duples[K].append(i)

In [32]:
duples.keys()

dict_keys([(0, 1), (0, 5), (0, 3), (3, 5), (0, 1, 5), (1, 5), (1, 2)])

In [36]:
duples[(0, 1)][:5]

[392, 903, 1002, 1158, 1216]

In [39]:
db[keys[392]]

{0: {'cnt': 1, 'list': [562]}, 1: {'cnt': 2, 'list': [383177, 389360]}}

지금 상황에서는 중복되는 level을 갖는 데이터가 정확하게 어떤 level인지 알 방법이 없음.

그러므로 수가 많은 쪽으로 가도록 하자

In [47]:
outdb = {"level": [], "text": []}
with tqdm(total=len(keys), ncols=100, file=sys.stdout) as t:
    for key in keys:
        d = db[key]
        if len(list(d.keys())) > 1:
            # 가장 개수가 많은 level을 선택
            cnts = sorted([(d[k]["cnt"], k) for k in d.keys()], reverse=True)

            # 만약 개수가 같은게 있다면 가장 level이 작은 것을 선택
            max_cnt = cnts[0][0]
            same_ks = list(filter(lambda x: x[0] == max_cnt, cnts))
            level = same_ks[-1][1]
        else:
            level = list(d.keys())[0]

        outdb["level"].append(level)
        outdb["text"].append(key)
        t.update()

100%|███████████████████████████████████████████████████| 152791/152791 [00:00<00:00, 931465.37it/s]


In [52]:
temp = defaultdict(int)
for level in outdb["level"]:
    temp[level] += 1

In [53]:
temp # 각 레벨별 데이터의 개수

defaultdict(int, {0: 73084, 1: 77270, 3: 710, 5: 1704, 2: 11, 4: 4, 6: 8})